In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload
%load_ext autoreload
%autoreload 2

import sys
from typing import List


import nltk
import torch
import datasets
import evaluate
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


sys.path.append("../")

In [2]:
1

1

In [9]:
try:
    a = 2 / 0
except:
    print("im here")


im here


In [3]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"

print(f"Device - {device}")

Device - mps


In [ ]:
# Load dataset that you want to tune your model on


dataset = datasets.load_dataset("c-s-ale/dolly-15k-instruction-alpaca-format")
rouge_metric = evaluate.load("rouge")

In [ ]:
task_types = ["open_"]

samples = []

for sample in dataset["train"]:
    if sample["category"] in task_types:
        samples.append(sample)

samples_to_tune_on = datasets.Dataset.from_list(samples)[:100]

In [ ]:
samples_to_tune_on.keys()

In [ ]:
samples_to_tune_on["input"]

In [ ]:
# Define your model

model_id = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

In [ ]:
type(model)

In [11]:
# Define your metric that you want to minimize or maximize on - should take in y_true & y_pred and return a dict of metrics


def my_metric(y_true: List, y_pred: List):
    def postprocess_text(preds: List, labels: List):
        """Postprocess text(preds & labels) to rouge format for evaluation"""
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
        return preds, labels

    """Get rouge score from y_true & y_pred"""
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)

    # TODO : fix metric
    return result["rouge2"]


# Example run on some random samples
my_metric(y_true=samples_to_tune_on["output"], y_pred=samples_to_tune_on["output"])

0.96

In [7]:
samples_to_tune_on

{'instruction': ['Why can camels survive for long without water?',
  "Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",
  'Who gave the UN the land in NY to build their HQ',
  'What is a polygon?',
  'Which episodes of season four of Game of Thrones did Michelle MacLaren direct?',
  'What individual has won the most Olympic gold medals in the history of the games?',
  'Which Dutch artist painted “Girl with a Pearl Earring”?',
  'What happens when the sun goes down?',
  'What is a verb?',
  'Who became king of Holland in 1806?',
  'Who played Billy the Kid in The Left Handed Gun',
  'why did Syd Barrett left the Pink Floyd?',
  'What is underwriting?',
  'Who saved Andromeda from the sea monster',
  'Who are \'\'The Lumières" ?',
  'When was the wheel invented?',
  'Are lilies safe for cats?',
  'What is Sunshine Recession?',
  'What is the currency in use in the Netherlands?',
  'In the series A Song of Ice and Fire, who is the founder of Ho

In [8]:
# Check dataset specific numbers to define tokenization limits

# For input

pd.Series(
    [
        len(
            tokenizer(sample, max_length=None, truncation=False, padding=False)[
                "input_ids"
            ]
        )
        for sample in samples_to_tune_on["instruction"]
    ]
).quantile([0.7, 0.8, 0.85, 0.9, 0.95])

0.70    13.00
0.80    16.00
0.85    17.00
0.90    19.00
0.95    21.05
dtype: float64

In [9]:
# For label
pd.Series(
    [
        len(
            tokenizer(sample, max_length=None, truncation=False, padding=False)[
                "input_ids"
            ]
        )
        for sample in samples_to_tune_on["output"]
    ]
).quantile([0.7, 0.8, 0.85, 0.9, 0.95])

Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


0.70     77.30
0.80    106.20
0.85    131.35
0.90    169.40
0.95    237.30
dtype: float64

In [10]:
# Check metrics on dataset without any optimization

model_input_tokenizer_kwargs = {
    "max_length": 20,
    "truncation": True,
    "padding": True,
}

generation_kwargs = {
    "temperature": 1,
    "max_new_tokens": 200,
    "num_beams": 3,
}

from llmsearch.utils.model_utils import infer_batches


model_output_before_tuning, latency = infer_batches(
    model=model,
    tokenizer=tokenizer,
    device=device,
    model_inputs=samples_to_tune_on["instruction"],
    model_input_tokenizer_kwargs=model_input_tokenizer_kwargs,
    generation_kwargs=generation_kwargs,
    batch_size=32,
)

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-vDz_II8E-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:686: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-vDz_II8E-py3.8/lib/python3.8/site-packages/transformers/generation/beam_search.py:358: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  sent_lengths_max = sent_lengths.max().item() + 1


NameError: name 'my_metric' is not defined

In [12]:
my_metric(y_true=samples_to_tune_on["output"], y_pred=model_output_before_tuning)

0.013292305726151015

In [13]:
# Run llm search

from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from llmsearch.wrappers.estimator import EstimatorWrapper

seed = 42


hyp_param_grid = {
    "max_new_tokens": [200],
    "num_beams": [3],
    "temperature": [1],
    "epsilon_cutoff": [3e-4, 6e-4, 9e-4, 0],
    "repetition_penalty": [0.3, 0.7, 1.0],
    "no_repeat_ngram_size": [0, 2, 3, 4],
}

model_estimator = EstimatorWrapper(
    model=model,
    tokenizer=tokenizer,
    device=device,
    batch_size=32,
    model_input_tokenizer_kwargs=model_input_tokenizer_kwargs,
)
scorer = make_scorer(score_func=my_metric, greater_is_better=True)

clf = RandomizedSearchCV(
    estimator=model_estimator,
    param_distributions=hyp_param_grid,
    n_iter=30,
    scoring=scorer,
    cv=5,
    random_state=seed,
    n_jobs=1,
    verbose=1,
)

In [14]:
clf.fit(X=samples_to_tune_on["instruction"], y=samples_to_tune_on["output"])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

RandomizedSearchCV(cv=5,
                   estimator=EstimatorWrapper(batch_size=32, device='mps',
                                              model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False...
                                              tokenizer=T5TokenizerFast(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>', '<extra_id_58>', '<extra_id_59>', '<extra_id_60>', '<extra_id_61>', '<extra_id_62>', '<extra_id_63>', '<extra_id_64>', '<extra_id_65>', '<extra_id_66>', '<extra_id_67>', '<extra_id_68>', '<extra_id_69>', '<extra_id_70>', '<extra_id_71>', '<extra_id_72>', '<extra_id_73>', '<extra_id_74>', '<extra_id_75>', '<extra_id_76>', '<extra_id_77>', '<extra_id_78>', '<extra_id_79>', '<extra_id_80>', '<extra_id_81>', '<extra_id_82>', '<extra_id_83>', '<extra_id_84>', '<extra_id_85>', '<extra_id_86>', '<extra_id_87>', '<extra_id_88>', '<extra_id_89>', '<extra_id_90>', '<extra_id_91>', '<extra_id_92>', '<extra_id_93>', '<extra_id_94>', '<extra_id_95>', '<extra_id_96>', '<extra_id_97>', '<extra_id_98>', '<extra_id_99>']})),
                   n_iter=30, n_jobs=1,
                   param_distributions={'epsilon_cutoff': [0.0003, 0.0006,
                                                           0.0009, 0],
                                        'max_new_tokens': [200],
                                        'no_repeat_ngram_size': [0, 2, 3, 4],
                                        'num_beams': [3],
                                        'repetition_penalty': [0.3, 0.7, 1.0],
                                        'temperature': [1]},
                   random_state=42, scoring=make_scorer(my_metric), verbose=1)

In [16]:
model_output_after_tuning, latency = infer_batches(
    model=model,
    tokenizer=tokenizer,
    device=device,
    model_inputs=samples_to_tune_on["instruction"],
    model_input_tokenizer_kwargs=model_input_tokenizer_kwargs,
    generation_kwargs=clf.best_params_,
    batch_size=32,
)

In [17]:
my_metric(y_true=samples_to_tune_on["output"], y_pred=model_output_before_tuning)

0.013292305726151015

In [18]:
my_metric(y_true=samples_to_tune_on["output"], y_pred=model_output_after_tuning)

0.03367519423588454

In [19]:
clf.best_params_

{'temperature': 1,
 'repetition_penalty': 0.7,
 'num_beams': 3,
 'no_repeat_ngram_size': 4,
 'max_new_tokens': 200,
 'epsilon_cutoff': 0}

In [20]:
model_output_after_tuning

['water is a source of a fungus and fungi that can survive for long without water and fungus that can live for a long time.',
 "Alice's parents have three daughters: Amy, Jessy, and what's Alice's mom's name?",
 'the United Nations',
 'polygon',
 'tv series vs. st. john s s tv vs st s n s r n tv',
 'samuel edwards',
 'samuel edwards',
 'the sun rises',
 'irrational adolescent adiolescence adilescence a dilescent dilad adlescent blunder ad ludicrous adalescence adicional adriolescent',
 'henry iii',
 'samuel edwards',
 'he was a saxophonist',
 'underwriting',
 'andromeda',
 "''The Lumières' ''",
 '1903–03–03 and 1904–03–04 and 1903–04–04 and 1906–04–03 and 1906–07–04 and 1911–04–07 and 1911–07–19 and 1911–1919–19 and 1912–1912–1914 and 1911–14–1914–1915–1915 and 1911–15–1914, 1911–1915, 1915–1918, 1915, 1912–1815, 1921–1918 and 1911–1819–1918–1921–1921, 1911–1818, 1911–21–1919, 1911–15, 1911–12, 1911–14, 1912, 1921, 1914–15, 1914–14, 1914–18, 1914–1919 and 1914–14–14–18–18–19–19–14–15–1

In [21]:
model_output_before_tuning

['water is a source of heat and moisture.',
 "Alice's parents have three daughters: Amy, Jessy, and what's Alice's parents have?",
 'the United Nations',
 'polygon',
 'tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series tv series',
 'samuel wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson wilson

In [22]:
for a, b, c in zip(
    samples_to_tune_on["output"], model_output_before_tuning, model_output_after_tuning
):
    print(f"Ground Truth - {a}")
    print(f"Model Output Before Tuning - {b}")
    print(f"Model Output After Tuning  - {c}")
    print()
    print("---" * 20)
    print()

Ground Truth - Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.
Model Output Before Tuning - water is a source of heat and moisture.
Model Output After Tuning  - water is a source of a fungus and fungi that can survive for long without water and fungus that can live for a long time.

------------------------------------------------------------

Ground Truth - The name of the third daughter is Alice
Model Output Before Tuning - Alice's parents have three daughters: Amy, Jessy, and what's Alice's parents have?
Model Output After Tuning  - Alice's parents have three daughters: Amy, Jessy, and what's Alice's mom's name?

------------------------------------------------------------

Ground Truth - John D Rockerfeller
Model Output Before Tuning - the United Nations
Model Output After Tuning  - the United Nations

------------------------------------------------------------

Ground Truth - A polygon is a form in Geometry.  It is a singl

In [24]:
samples_to_tune_on["output"]

['Tope',
 'Gudok is string, Cantaro is percussion.',
 'The useful objects for transportation in the list are train, plane, bicyle and scooter.',
 'Zurna is woodwind, Panduri is string.',
 'Shamisen is string, Kpanlogo is percussion.',
 'blue, yellow, green, red, purple, orange',
 'Eminem, 50 Cent',
 'The classification is as follows:\n- Soccer: goalie gloves, corner flag\n- Basketball: shooting sleeve\n- Football: penalty flag, kicking tee',
 'Giant Tortoise is alive, Palaeophis is extinct.',
 'Viola toeria is string, Samphor is percussion.',
 'Inside a house you can find a chair, a table and a microwave. Cars, clouds and parks are found outside of houses.',
 'Dark-colored beers: porter, stout, amber\nLight-colored beers: pilsner, lager',
 'avocados, kiwi, pairs, blueberries, green beans, kale, cabbage',
 'Guitar - String\nViolin - String\npiano - keyboard\nharmonium - keyboard\ncello - string\naccordion - keyboard\nbanjo - string',
 'Bacon, eggs, sausages, beans, toast, mushrooms, tom